In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
data = pd.read_csv('../../../data/Personal_Victimization_Original.csv')
data.head()

,idper,yearq,year,ager,sex,hispanic,race,race_ethnicity,hincome1,hincome2,...,weapcat,injury,serious,treatment,offenderage,offendersex,offtracenew,wgtviccy,series,newwgt
0,150102759544999999999913501,1993.1,1993,6,2,2,1,1,4,-1,...,5.0,0,1,0.0,3,1,-1,8442.41411,1,8442.41411
1,150123750403999999999911602,1993.1,1993,4,2,2,1,1,4,-1,...,0.0,0,1,0.0,1,1,-1,5158.88511,1,5158.88511
2,150166998771999999999913502,1993.1,1993,3,2,2,2,2,2,-1,...,0.0,1,3,1.0,2,1,-1,3006.10621,1,3006.10621
3,150283624848999999999931601,1993.1,1993,2,1,2,1,1,6,-1,...,0.0,0,1,0.0,3,1,-1,4533.87724,1,4533.87724
4,150283624848999999999931601,1993.1,1993,2,1,2,1,1,6,-1,...,0.0,0,1,0.0,3,1,-1,4533.87724,1,4533.87724


In [ ]:
# Modelo simple
model_baseline = Sequential([
    Dense(32, activation='relu', input_shape=(16,)),
    Dense(7, activation='softmax')  # 7 clases
])

In [ ]:
model_baseline.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history_crime = model_baseline.fit(X_train_scaled, y_train,
                                   validation_split=0.2,
                                   epochs=20,
                                   batch_size=32,
                                   verbose=1)

In [ ]:
# Evaluar train

train_loss, train_acc = model_baseline.evaluate(X_train_scaled, y_train, verbose=0)

print(f"Train Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Train Loss: {train_loss:.4f}")
print("="*50)

print("\n Baseline del azar (3 clases): 33.3%")

In [ ]:
# Evaluar test

test_loss, test_acc = model_baseline.evaluate(X_test_scaled, y_test, verbose=0)

print("="*50)
print(" RESULTADOS BASELINE - Tipos crimen")
print("="*50)
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print("="*50)

In [ ]:
# Visualización de curvas de entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Crear rango de epochs (1 a 20, no 0 a 19)
epochs = range(1, len(history_crime.history['loss']) + 1)

# Loss
ax1.plot(epochs, history_crime.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(epochs, history_crime.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Curvas de Loss')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(epochs, history_crime.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(epochs, history_crime.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Curvas de Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de confusión
y_pred_crime = model_baseline.predict(X_test_scaled)
y_pred_crime_classes = np.argmax(y_pred_crime, axis=1)

cm = confusion_matrix(y_test, y_pred_crime_classes)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay'],
            yticklabels=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay'])
plt.title('Matriz de Confusión - Tipos judía')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\n Classification Report:")
print(classification_report(y_test, y_pred_crime_classes, 
                          target_names=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay']))